<a href="https://colab.research.google.com/github/kimhwijin/TensorflowWithKeras/blob/master/CNN/Deep_CIFAR_10_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, models, layers, regularizers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [8]:
EPOCHS = 15
NUM_CLASSES = 10
INPUT_SHAPE = (32, 32, 3)
def load_data():
  (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
  
  x_train = x_train.reshape(50000, 32, 32, 3)
  x_test = x_test.reshape(10000, 32, 32, 3)
  
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')

  #regularization

  #x_train모든 값의 평균
  mean = np.mean(x_train, axis=(0,1,2,3))
  std = np.std(x_train, axis=(0,1,2,3))
  x_train = (x_train - mean) / (std + 1e-7)
  x_test = (x_test - mean) / (std + 1e-7)

  y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
  y_test = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)

  return (x_train, y_train), (x_test, y_test)

In [7]:
#심층 cnn
def build_deep_model(input_shape, num_classes):

  model = models.Sequential()
  #첫번째 블록
  model.add(layers.Convolution2D(32, (3,3), padding='same', input_shape=input_shape,activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Convolution2D(32, (3,3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.Dropout(0.2))

  model.add(layers.Convolution2D(64, (3,3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Convolution2D(64, (3,3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.Dropout(0.3))

  model.add(layers.Convolution2D(128, (3,3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Convolution2D(128, (3,3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.Dropout(0.4))

  model.add(layers.Flatten())
  model.add(layers.Dense(num_classes, activation='softmax'))

  summary = model.summary()
  
  return model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

(x_train, y_train), (x_test, y_test) = load_data()
#이미지 보강
datagen = ImageDataGenerator(
    rotation_range = 30, #30도 회전
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip=True,
)
datagen.fit(x_train)

model = build_deep_model(INPUT_SHAPE, NUM_CLASSES)
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

In [ ]:
batch_size = 64
model.fit(x_train,y_train, batch_size=batch_size, epochs=EPOCHS, validation_data=(x_test, y_test))

model_json = model.to_json()
with open('model.json', 'w') as json_file:
  json_file.write(model_json)
model.save_weights('model.h5')

score = model.evaluate(x_test, y_test, batch_size=batch_size)
print('\nTest Score : ',score[0])
print('\nTest Accuracy : ',score[1])

In [ ]:
#with data generator
batch_size = 64
model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size), epochs=EPOCHS, verbose=1, validation_data=(x_test,y_test))

model_json = model.to_json()
with open('model.json', 'w') as json_file:
  json_file.write(model_json)
model.save_weights('model.h5')

#test
score = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f ' %(score[1]*100, score[0]))